In [1]:
import sys
sys.path.append('../scripts/')
from kf import *

In [2]:
def make_ax():
    fig = plt.figure(figsize=(4, 4))
    ax = fig.add_subplot(111)
    ax.set_aspect('equal')
    ax.set_xlim(-5, 5)
    ax.set_ylim(-5, 5)
    ax.set_xlabel("X", fontsize=10)
    ax.set_ylabel("Y", fontsize=10)
    return ax

def draw_trajectory(xs, ax):
    poses = [xs[s] for s in range(len(xs))]
    ax.scatter([e[0] for e in poses], [e[1] for e in poses], s=5, marker=".", color="black")
    ax.plot([e[0] for e in poses], [e[1] for e in poses], linewidth=0.5, color="black")
    
def draw_observation(xs, zlist, ax):
    for s in range(len(xs)):
        if s not in zlist:
            continue
            
        for obs in zlist[s]:
            x, y, theta = xs[s]
            ell, phi = obs[1][0], obs[1][1]
            mx = x + ell*math.cos(theta + phi)
            my = y + ell*math.sin(theta + phi)
            ax.plot([x, mx], [y, my], color="pink", alpha=0.5)

def draw_edges(edges, ax):
    for e in edges:
        ax.plot([e.x1[0], e.x2[0]], [e.x1[1], e.x2[1]], color="red", alpha=0.5)

def draw_landmarks(ms, ax):
    ax.scatter([ms[k][0] for k in ms], [ms[k][1] for k in ms], s=100, marker="*", color="blue", zorder=100)
        
def draw(xs, zlist, edges, ms={}):
    ax = make_ax()
    draw_observation(xs, zlist, ax)
    draw_trajectory(xs, ax)
    draw_landmarks(ms, ax)
    plt.show()

In [3]:
def read_data():
    hat_xs = {}
    zlist = {}
    delta = 0.0
    us = {}
    
    with open("log2.txt") as f:
        for line in f.readlines():
            tmp = line.rstrip().split()
            
            step = int(tmp[1])
            if tmp[0] == "x":
                hat_xs[step] = np.array([float(tmp[2]), float(tmp[3]), float(tmp[4])]).T
            elif tmp[0] == "z":
                if step not in zlist:
                    zlist[step] = []
                zlist[step].append((int(tmp[2]), np.array([float(tmp[3]), float(tmp[4]), float(tmp[5])]).T))
            elif tmp[0] == "delta":
                delta = float(tmp[1])
            elif tmp[0] == "u":
                us[step] = np.array([float(tmp[2]), float(tmp[3])]).T
                
        return hat_xs, zlist, us, delta

In [4]:
class ObsEdge:
    def __init__(self, t1, t2, z1, z2, xs, sensor_noise_rate=[0.14, 0.05, 0.05]):
        assert z1[0] == z2[0]
        
        self.t1, self.t2 = t1, t2
        self.x1, self.x2 = xs[t1], xs[t2]
        self.z1, self.z2 = z1[1], z2[1]
        
        s1 = math.sin(self.x1[2] + self.z1[1])
        c1 = math.cos(self.x1[2] + self.z1[1])
        s2 = math.sin(self.x2[2] + self.z2[1])
        c2 = math.cos(self.x2[2] + self.z2[1])
        
        hat_e = self.x2 - self.x1 + np.array([
            self.z2[0]*c2 - self.z1[0]*c1,
            self.z2[0]*s2 - self.z1[0]*s1,
            self.z2[1] - self.z2[2] - self.z1[1] + self.z1[2]
        ])
        while hat_e[2] >= math.pi: hat_e[2] -= math.pi*2
        while hat_e[2] < - math.pi: hat_e[2] += math.pi*2
            
        Q1 = np.diag([(self.z1[0]*sensor_noise_rate[0])**2, sensor_noise_rate[1]**2, sensor_noise_rate[2]**2])
        R1 = - np.array([[c1, -self.z1[0]*s1, 0],
                        [s1, self.z1[0]*c1, 0],
                        [0, 1, -1]])
        
        Q2 = np.diag([(self.z2[0]*sensor_noise_rate[0])**2, sensor_noise_rate[1]**2, sensor_noise_rate[2]**2])
        R2 = np.array([[c2, -self.z2[0]*s2, 0],
                      [s2, self.z2[0]*c2, 0],
                      [0, 1, -1]])
        
        Sigma = R1.dot(Q1).dot(R1.T) + R2.dot(Q2).dot(R2.T)
        Omega = np.linalg.inv(Sigma)
        
        B1 = - np.array([[1, 0, -self.z1[0]*s1],
                        [0, 1, self.z1[0]*c1],
                        [0, 0, 1]])
        B2 = np.array([[1, 0, -self.z2[0]*s2],
                      [0, 1, self.z2[0]*c2],
                      [0, 0, 1]])
        
        self.omega_upperleft = B1.T.dot(Omega).dot(B1)
        self.omega_upperright = B1.T.dot(Omega).dot(B2)
        self.omega_bottomleft = B2.T.dot(Omega).dot(B1)
        self.omega_bottomright = B2.T.dot(Omega).dot(B2)
        
        self.xi_upper = - B1.T.dot(Omega).dot(hat_e)
        self.xi_bottom = - B2.T.dot(Omega).dot(hat_e)

In [5]:
class MotionEdge:
    def __init__(self, t1, t2, xs, us, delta, motion_noise_stds={"nn": 0.19, "no": 0.001, "on": 0.13, "oo": 0.2}):
        self.t1, self.t2 = t1, t2
        self.hat_x1, self.hat_x2 = xs[t1], xs[t2]
        
        nu, omega = us[t2]
        if abs(omega) < 1e-5: omega = 1e-5
            
        M = matM(nu, omega, delta, motion_noise_stds)
        A = matA(nu, omega, delta, self.hat_x1[2])
        F = matF(nu, omega, delta, self.hat_x1[2])
        
        self.Omega = np.linalg.inv(A.dot(M).dot(A.T) + np.eye(3)*0.0001)
        
        self.omega_upperleft = F.T.dot(self.Omega).dot(F)
        self.omega_upperright = -F.T.dot(self.Omega)
        self.omega_bottomleft = -self.Omega.dot(F)
        self.omega_bottomright = self.Omega
        
        x2 = IdealRobot.state_transition(nu, omega, delta, self.hat_x1)
        self.xi_upper = F.T.dot(self.Omega).dot(self.hat_x2 - x2)
        self.xi_bottom = - self.Omega.dot(self.hat_x2 - x2)

In [6]:
import itertools
def make_edges(hat_xs, zlist):
    landmark_keys_zlist = {}
    
    for step in zlist:
        for z in zlist[step]:
            landmark_id = z[0]
            if landmark_id not in landmark_keys_zlist:
                landmark_keys_zlist[landmark_id] = []
                
            landmark_keys_zlist[landmark_id].append((step, z))
            
    edges = []
    for landmark_id in landmark_keys_zlist:
        step_pairs = list(itertools.combinations(landmark_keys_zlist[landmark_id], 2))
        edges += [ObsEdge(xz1[0], xz2[0], xz1[1], xz2[1], hat_xs) for xz1, xz2 in step_pairs]
        
    return edges, landmark_keys_zlist

In [7]:
def add_edge(edge, Omega, xi):
    f1, f2 = edge.t1*3, edge.t2*3
    t1, t2 = f1 + 3, f2 + 3
    Omega[f1:t1, f1:t1] += edge.omega_upperleft
    Omega[f1:t1, f2:t2] += edge.omega_upperright
    Omega[f2:t2, f1:t1] += edge.omega_bottomleft
    Omega[f2:t2, f2:t2] += edge.omega_bottomright
    xi[f1:t1] += edge.xi_upper
    xi[f2:t2] += edge.xi_bottom

In [8]:
hat_xs, zlist, us, delta = read_data()
dim = len(hat_xs)*3

for n in range(1, 10000):
    edges, _ = make_edges(hat_xs, zlist)
    
    for i in range(len(hat_xs)-1):
        edges.append(MotionEdge(i, i+1, hat_xs, us, delta))
    
    Omega = np.zeros((dim, dim))
    xi = np.zeros(dim)
    Omega[0:3, 0:3] += np.eye(3)*1000000
    
    for e in edges:
        add_edge(e, Omega, xi)
        
    delta_xs = np.linalg.inv(Omega).dot(xi)
    
    for i in range(len(hat_xs)):
        hat_xs[i] += delta_xs[i*3:(i+1)*3]
        
    diff = np.linalg.norm(delta_xs)
    print("{}回目の繰り返し: {}".format(n, diff))
    if diff < 0.01:
        draw(hat_xs, zlist, edges)
        break

1回目の繰り返し: 21.995064347490228
2回目の繰り返し: 10.99094883836067
3回目の繰り返し: 4.416349352647396
4回目の繰り返し: 2.46857119541007
5回目の繰り返し: 1.4733550155735713
6回目の繰り返し: 0.8952127113296969
7回目の繰り返し: 0.5513205250981944
8回目の繰り返し: 0.34313605904505545
9回目の繰り返し: 0.21536685281096946
10回目の繰り返し: 0.1362063304211353
11回目の繰り返し: 0.08685602385669092
12回目の繰り返し: 0.05596733867461984
13回目の繰り返し: 0.036585706871567356
14回目の繰り返し: 0.02440634283681976
15回目の繰り返し: 0.01674712621932212
16回目の繰り返し: 0.01192972140759084
17回目の繰り返し: 0.008900723511957335


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
_, zlist_landmark = make_edges(hat_xs, zlist)
zlist_landmark

{1: [(0, (1, array([ 2.07287078, -0.06625311,  3.19759195]))),
  (1, (1, array([ 1.54205905, -0.22987778,  3.09636591]))),
  (2, (1, array([ 1.04440744, -0.6928954 ,  2.9071886 ]))),
  (19, (1, array([4.83119281, 0.93735063, 2.84667918]))),
  (20, (1, array([4.81241787, 0.73718253, 2.92344042]))),
  (21, (1, array([4.60692894, 0.61401579, 3.08635722]))),
  (22, (1, array([4.62526351, 0.42908842, 3.10154347]))),
  (23, (1, array([ 3.12321429,  0.27104338, -3.06009494]))),
  (24, (1, array([ 2.57877721, -0.0331555 , -3.04774051]))),
  (25, (1, array([ 2.02810375, -0.39849458, -3.15768921]))),
  (26, (1, array([ 1.43574948, -0.79513186,  2.96460555]))),
  (42, (1, array([4.43217127, 0.94544265, 3.10760231]))),
  (43, (1, array([ 3.64403759,  0.65614388, -2.93569417]))),
  (44, (1, array([ 3.03212654,  0.56674505, -2.90671564]))),
  (45, (1, array([ 2.58924412,  0.41875963, -2.86290111]))),
  (46, (1, array([ 2.19361018,  0.25891087, -2.7258075 ]))),
  (47, (1, array([ 1.1524109 , -0.27266

In [10]:
class MapEdge:
    def __init__(self, t, z, head_t, head_z, xs, sensor_noise_rate=[0.14, 0.05, 0.05]):
        self.x = xs[t]
        self.z = z
        
        self.m = self.x + np.array([
            z[0]*math.cos(self.x[2] + z[1]),
            z[0]*math.sin(self.x[2] + z[1]),
            - xs[head_t][2] + z[1] - head_z[1] - z[2] + head_z[2]
        ]).T
        
        while self.m[2] >= math.pi: self.m[2] -= math.pi*2
        while self.m[2] < -math.pi: self.m[2] += math.pi*2
            
        Q1 = np.diag([(self.z[0]*sensor_noise_rate[0])**2, sensor_noise_rate[1]**2, sensor_noise_rate[2]**2])
        
        s1 = math.sin(self.x[2] + self.z[1])
        c1 = math.cos(self.x[2] + self.z[1])
        R = np.array([[-c1, self.z[0]*s1, 0],
                     [-s1, -self.z[0]*c1, 0],
                     [0, -1, 1]])
        
        self.Omega = R.dot(Q1).dot(R.T)
        self.xi = self.Omega.dot(self.m)

In [11]:
ms = {}
for landmark_id in zlist_landmark:
    edges = []
    head_z = zlist_landmark[landmark_id][0]
    for z in zlist_landmark[landmark_id]:
        edges.append(MapEdge(z[0], z[1][1], head_z[0], head_z[1][1], hat_xs))
    
    Omega = np.zeros((3, 3))
    xi = np.zeros(3)
    for e in edges:
        Omega += e.Omega
        xi += e.xi
    
    ms[landmark_id] = np.mean([e.m for e in edges], axis=0)
    
draw(hat_xs, zlist, edges, ms)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
actual_pos = [(-4, 2), (2, -3), (3, 3), (0, 4), (1, 1), (-3, -1)]
ms

{1: array([ 2.20475264, -3.27134788, -0.05630025]),
 4: array([ 1.78002204,  0.78279963, -0.01663473]),
 2: array([ 4.17351179,  2.52080129, -0.01482064]),
 3: array([1.15915515e+00, 3.94480243e+00, 2.07124888e-03]),
 0: array([-3.09984999,  2.13081618, -0.0182941 ]),
 5: array([-2.57112019, -1.1212301 ,  0.00675926])}

In [13]:
def distance(id1, id2, ms):
    m1, m2 = ms[id1], ms[id2]
    return math.sqrt((m1[0]-m2[0])**2 + (m1[1] - m2[1])**2)